In [29]:
import csv
import numpy as np
import matplotlib.pyplot as plt

mu_depth, sigma_depth = 3, 1.5
mu_delta, sigma_delta = 0.0001, 0.1

fixingfactor = 0.2
delta = []
depth = []
output = []
flag = []
for k in range(1000):
    flag.append(0)
# temp2 = []
depth.append(np.random.normal(mu_depth, sigma_depth, 1000))
# depth.append(temp2)
for i in range(100):    
    delta.append(np.random.normal(mu_delta, sigma_delta, 1000))
    temp = []
    temp2 = []
    count = 0
     # fixing factor
        
    for j in range(1000):
        if flag[j] > 0:
            flag[j] -= 1
        if flag[j] <  0:
            depth[i][j] = 4
            flag[j] = 0
        if depth[i][j] - i*delta[i][j] >= 0:
            temp.append(depth[i][j] - i*delta[i][j])
        else:
            temp.append(0)
        if depth[i][j] - i*delta[i][j] < 0.5:
            temp2.append(1)
            flag[j] = np.random.randint(5,10) 
            count += 1
        else:
            temp2.append(0)
    
        
    depth.append(temp)
    output.append(temp2)
    print(count)

print(output)
# print("delta =")
# print(delta)
# print("depth =")
# print(depth)
# count = 0
# for i in range(1000):
#     if depth[i] < 0:
#         count = count + 1
#print(depth)

48
45
48
51
55
68
67
81
99
89
119
121
138
137
159
172
185
174
191
190
189
198
190
194
186
167
188
158
154
154
161
162
175
160
155
138
157
152
169
160
146
154
133
130
116
129
131
126
132
134
138
150
124
118
120
131
131
135
121
116
128
125
118
105
103
128
111
119
100
96
108
93
96
100
109
112
104
107
122
115
113
113
117
112
96
105
113
101
99
113
100
117
112
112
113
105
101
95
97
96
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [5]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)


ImportError: No module named tensorflow